In [ ]:
import pandas as pd
import requests
from tqdm import tqdm_notebook
import pandas_profiling

In [ ]:
df = pd.read_csv("./report-2018-11-22T2156.csv").dropna(axis=1, how='all')

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
#df.head()

In [ ]:
# https://genderize.io/#overview
r = requests.get("https://api.genderize.io", params={"name": "Daryna"})
r.json()

In [ ]:
genders = []
for name in tqdm_notebook(list(df["First Name"])):
    r = requests.get("https://api.genderize.io", params={"name": name})
    genders.append(r.json())

In [ ]:
genders_df = pd.DataFrame(genders)
df['Gender'] = genders_df['gender']
df['GenderP'] = genders_df['probability']

In [ ]:
df = df.loc[lambda df: df['Order Type'] != "Abandoned"]

In [ ]:
# removing booth staff and tutorials
df = df \
  .loc[lambda df: df['Discount'].apply(lambda x: pd.isnull(x) or 'booth' not in x.lower())] \
  .loc[lambda df: df['Ticket Type'].apply(lambda x: "Tutorial" not in x)]

In [ ]:
df['Quantity'].value_counts()

In [ ]:
df.columns

In [ ]:
df.groupby(['Ticket Type', 'Order Type']).size()

In [ ]:
# privacy!
df = df.loc[:, ['Attendee Status', 'Order Date', 'Check-In Date', 'IP Location',  'Would you like a T-Shirt?',
                'What size?', 'Ticket Type', 'Gender', 'GenderP']]  
# no 'Job Title' this time due to its disclaimer

In [ ]:
#df.head()

In [ ]:
#df['Ticket Type'].value_counts()

In [ ]:
df['Participant Type'] = df['Ticket Type'].apply(lambda x: 'Individual' if 'Individual' in x else 'Corporate')
def call_type(x):
    if 'Early Bird' in x:
        return 'Early Bird'
    elif 'Final Call' in x:
        return 'Final Call'
    else:
        return "Regular"

df['Call Type'] = df['Ticket Type'].apply(call_type)

In [ ]:
df['Participant Type'].value_counts()

In [ ]:
df['Call Type'].value_counts()

In [ ]:
df['Country'] = df['IP Location'].apply(lambda x: x.split(", ")[-1])
df['City'] = df['IP Location'].apply(lambda x: x.split(", ")[0] if "," in x else "")

In [ ]:
df = df.drop(['IP Location', 'Ticket Type'], axis=1)

In [ ]:
df = df.sort_values(by="Order Date")

In [ ]:
df.head()

In [ ]:
df.to_csv("pdwc2018_anonym.csv", index=False)

In [ ]:
#pandas_profiling.ProfileReport(df)